In [2]:
import os
import numpy as np
import emu.neuralynx_io as nlx
import pandas as pd
import datetime


In [2]:
ncs = neuralynx_io.load_ncs(os.path.expanduser('~/.emu/pt1/sEEG/raw/CSC6.ncs'))

/home/elijahc/projects/emu/notebooks/src/neuralynx_io.py:78: UserWarning: Unable to parse original file path from Neuralynx header: -FileType NCS
  warnings.warn('Unable to parse original file path from Neuralynx header: ' + hdr_lines[1])
/home/elijahc/projects/emu/notebooks/src/neuralynx_io.py:130: UserWarning: Unable to parse time string from Neuralynx header: -FileVersion 3.4
  warnings.warn('Unable to parse time string from Neuralynx header: ' + time_string)
/home/elijahc/projects/emu/notebooks/src/neuralynx_io.py:130: UserWarning: Unable to parse time string from Neuralynx header: -FileUUID d0c2964e-e524-40e5-a986-ca069f38c58c
  warnings.warn('Unable to parse time string from Neuralynx header: ' + time_string)
/home/elijahc/projects/emu/notebooks/src/neuralynx_io.py:92: UserWarning: Unable to parse parameter line from Neuralynx header: -ProbeName
  warnings.warn('Unable to parse parameter line from Neuralynx header: ' + line)
/home/elijahc/projects/emu/notebooks/src/neuralynx_io.p

In [9]:
header = ncs['header']
pp.pprint(header)

{   'ADBitVolts': '0.000000030517578125000001',
    'ADChannel': '5',
    'ADMaxValue': '32767',
    'AcqEntName': 'CSC6',
    'DSPHighCutFilterEnabled': 'True',
    'DSPLowCutFilterEnabled': 'True',
    'DspDelayCompensation': 'Enabled',
    'DspFilterDelay_µs': '3984',
    'DspHighCutFilterType': 'FIR',
    'DspHighCutFrequency': '2000',
    'DspHighCutNumTaps': '256',
    'DspLowCutFilterType': 'DCO',
    'DspLowCutFrequency': '0.1',
    'DspLowCutNumTaps': '0',
    'InputInverted': 'True',
    'InputRange': '1000',
    'NumADChannels': '1',
    'OriginalFileName': '"D:\\PegasusData\\2019-07-30_11-04-51\\CSC6.ncs"',
    'RecordSize': '1044',
    'SamplingFrequency': '8000',
    'SessionUUID': '0fbe95c6-940b-47a6-b5b7-85ebd4cbc782',
    'TimeClosed': 'leUUID d0c2964e-e524-40e5-a986-ca069f38c58c',
    'TimeClosed_dt': None,
    'TimeOpened': 'leVersion 3.4',
    'TimeOpened_dt': None}


In [10]:
dat = ncs['data']
len(dat)

115204608

In [12]:
dat

array([ 3.11279297,  2.34985352,  2.99072266, ..., 48.06518555,
       45.71533203, 46.35620117])

In [11]:
len(dat)/8000/60/60

4.000159999999999